In [47]:
# Office and Excel python modules
import docx
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
from docx.shared import RGBColor

import xlsxwriter

# For working with images and saving to local filesystem
import shutil
import requests
import sys
from PIL import Image, ImageDraw, ImageFont

# For Excel / chart making
import win32com.client as win32
from docx.shared import Inches
from win32com.client import constants as c


import pandas as pd
import numpy as np

In [36]:
def initdoc(template):
    document = Document(template)
    # Set the styles of the Word doc
    styles = document.styles

    header = styles.add_style('Main', WD_STYLE_TYPE.PARAGRAPH)
    header.font.name = 'Futura BdCn BT'
    header.font.size = Pt(28)
    header.font.color.rgb = RGBColor(0, 77, 132)

    section = styles.add_style('Section', WD_STYLE_TYPE.PARAGRAPH)
    section.font.name = 'Futura BdCn BT'
    section.font.size = Pt(16)
    section.font.color.rgb = RGBColor(0, 77, 132)

    style = document.styles['SEIA Bullet 1']
    font = style.font
    font.name = 'Futura Std Book'
    font.size = Pt(10.5)
    return document

In [93]:
def topten(rank10):
    if (rank10 == 1):
        t_rank = "first"
    elif (rank10 == 2):
        t_rank = "second"
    elif (rank10 == 3):
        t_rank = "third"
    elif (rank10 == 4):
        t_rank = "fourth"
    elif (rank10 == 5):
        t_rank = "fifth"
    elif (rank10 == 6):
        t_rank = "sixth"
    elif (rank10 ==7):
        t_rank = "seventh"
    elif (rank10 == 8):
        t_rank = "eighth"
    elif (rank10 == 9):
        t_rank = "ninth"
    else:
        t_rank = "ERROR topten"
    return t_rank

def doubledigits(rank20):
    if(rank20 == 2):
        dd = "twenty"
    elif(rank20 == 3):
        dd = "thirty"
    elif(rank20 == 2):
        dd = "twenty"
    elif(rank20 == 3):
        dd = "thirty"
    elif(rank20 == 2):
        dd = "forty"
    elif(rank20 == 3):
        dd = "fifty"
    else:
        dd = "ERROR doubledigits"
    return dd
        
def rankTorank(arank):
    if(arank > 0 and arank < 10):
        text_rank = topten(arank)
    elif (arank < 20 and arank >= 10):
        if (arank == 10):
            text_rank = "tenth"
        elif (arank == 11):
            text_rank = "eleventh"
        elif (arank == 12):
            text_rank = "twelfth"
        elif (arank == 13):
            text_rank = "thirteenth"
        elif (arank == 14):
            text_rank = "fourteenth"
        elif (arank == 15):
            text_rank = "fifteenth"
        elif (arank == 16):
            text_rank = "sixteenth"
        elif (arank == 17):
            text_rank = "seventeenth"
        elif (arank == 18):
            text_rank = "eighteenth"
        elif (arank == 19):
            text_rank = "nineteenth"
        else:
            text_rank = "ERROR (gte0lt20)"
    elif(arank < 100 and arank >= 20):
        bigdigit = np.round(arank / 10, decimals=0)
        lildigit = arank % 10
        if(lildigit == 0):
            f_text = doubledigits(bigdigit)
            # remove the "-y" from the back of the word
            f_text = f_text[:-1]
            text_rank = f_text + "ieth"
        else:
            f_text = doubledigits(bigdigit)
            b_text = topten(lildigit)
            text_rank = f_text + " " + b_text
    else:
        text_rank = "ERROR (gte20)"
    return text_rank


In [38]:
def graf1(document, state, cos, jobs, mfrs, mfr_facs, installers, devs, distribs, others):
    t = document.add_paragraph('Solar Spotlight: ' + state, style="Main")
    # Section heading
    s1 = document.add_paragraph('At a glance', style="Section")
    #print(cos)
    # First paragraph
    graf1 = document.add_paragraph("There are currently more than ", style="SEIA Bullet 1")
    graf1.add_run(str(cos) + " solar companies").bold = True
    graf1.add_run(" at work throughout the value chain in" + state + ",")
    graf1.add_run(" employing " + str(jobs) + " people.").bold = True
    graf1.add_run("These companies provide a wide variety of solar products and services ranging from solar system installations to the manufacturing of components used in photovoltaic panels. These companies can be broken down across the following categories: ")
    graf1.add_run(str(mfrs) + " manufacturers, ")
    graf1.add_run(str(mfr_facs) + " manufacturing facilities, ")
    graf1.add_run(str(installers) + " contractor/installers, ")
    graf1.add_run(str(devs) + " project developers, ")
    graf1.add_run(str(distribs) + " distributors and ")
    graf1.add_run(str(others) + " engaged in other solar activities including financing, engineering, and legal support.")
    #document.save('J:/SEIA/codebin/Factsheets/test2016122_graf1test.docx')
    return

In [97]:
def makeChart(data):    
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.Visible = False
    #excel = win32.Dispatch("Excel.Application")
#excel.Visible = False
# Open the workbook
    
    wb = excel.Workbooks.Open(data[0])
    # Get the sheet
    ws = wb.Worksheets('Sheet1')         # alternatively Work
    state = data[1]
    newdata = data[2]
    print(newdata)
    ws.Cells(1,1).Value='Year'
    ws.Cells(1,2).Value= state + ' Annual Capacity (MW)'
    ws.Cells(2,1).Value='2007'
    ws.Cells(2,2).Value=10
    ws.Cells(3,1).Value='2008'
    ws.Cells(3,2).Value=50
    ws.Cells(4,1).Value='2009'
    ws.Cells(4,2).Value=100
    ws.Cells(5,1).Value='2010'
    ws.Cells(5,2).Value=newdata[2010].values[0]
    ws.Cells(6,1).Value='2011'
    ws.Cells(6,2).Value = newdata[2011].values[0]
    #if(newdata[2011].empty == False):
    #    ws.Cells(6,2).Value=newdata[2011].values[0]
    #else:
    #    ws.Cells(6,2).Value=0
    ws.Cells(7,1).Value='2012'
    ws.Cells(7,2).Value=newdata[2012].values[0]
    ws.Cells(8,1).Value='2013'
    ws.Cells(8,2).Value=newdata[2013].values[0]
    ws.Cells(9,1).Value='2014'
    ws.Cells(9,2).Value=newdata[2014].values[0]
    ws.Cells(10,1).Value='2015'
    ws.Cells(10,2).Value=newdata[2015].values[0]
    ws.Cells(11,1).Value='2016E'
    ws.Cells(11,2).Value=newdata[2016].values[0]
    
    
    
    chart = ws.Shapes.AddChart().Select()
    #series = chart.SeriesCollection().NewSeries()
    #Xval = ws.Range("A1:B11")
    #series.XValues = Xval
    
    #excel.ActiveChart.ChartType = c.Columns
    excel.ActiveChart.SetSourceData(Source=ws.Range("A1:B11"))
    excel.ActiveChart.Legend.Delete()
    
    #series = chart.SeriesCollection(1)
    #series.XValues = ws.Range("B1:K1")
    #series.Values = ws.Range("B2:K2")
    #series.Name = state + ' Annual Capacity (MW)'
    
    
    #cs = wb.Worksheets(state)
    #c = wb.Worksheets(state).ChartObjects(1)
    #ws.Add(chart)
    c = ws.ChartObjects(1)
    c.CopyPicture
    #print(c)
    # Get the first chart
    #c = ws.ChartObjects(1)
    # Copy the picture object
    #c.CopyPicture
    #chart.CopyPicture
    # Export said object to location provided
    c.Chart.Export("J:/SEIA/codebin/Factsheets/Q42016/charts/" + data[1] + "_chart.jpg")
    
    #chart.Chart.Export("J:/SEIA/codebin/Factsheets/charts/" + data[1] + "_chart.jpg")
    # Close the workbook and the Excel engine
    wb.Close(False)
    excel.Quit()
    # return a string of the location of the image 
    return "J:/SEIA/codebin/Factsheets/Q42016/charts/" + data[1] + "_chart.jpg"


In [40]:
def graf2(document, state, q32016, qq, yy, rank):
    # Quarterly Capacity and % growth figures (q/q and y/y)
    graf2 = document.add_paragraph("", style="SEIA Bullet 1")
    graf2.add_run(str(q32016) + " MW of solar energy were installed").bold = True
    graf2.add_run(" in " + state + " in the fourth quarter of 2016. This marks a " + str(qq) + " increase over the last quarter and a " + str(yy) +" increase over the same quarter last year. " + state + " ranked " + str(rank) + " nationally in fourth quarter installations.")
    return
    
def graf3(document, state, cap_total, cap_res, cap_nonres, cap_utility, rank):
    # Annual capacity w/ state ranking and capacity by sector
    graf3 = document.add_paragraph("In 2015, " + state, style="SEIA Bullet 1")
    graf3.add_run(" installed " + str(cap_total) + " MW of solar electric capacity,").bold = True
    graf3.add_run("ranking it " + str(rank) + " nationally. Of this capacity, " 
                  + str(cap_res) + " MW were residential, " 
                  + str(cap_nonres) + " MW were commercial and "
                  + str(cap_utility) + " MW were utility-scale.")
    return

def graf4(document, state, cap_total, rank, hp):
    # Cumulative solar and homes powered
    graf4 = document.add_paragraph("The ", style="SEIA Bullet 1")
    graf4.add_run(str(cap_total) + " MW of solar energy currently installed").bold = True
    graf4.add_run(" in " + state + " ranks the state " + str(rank) + " in the country in installed solar capacity.  There is enough solar energy installed in the state to power " 
                  + str(hp) + " homes.")
    return
    
def othergrafs(document, state, data):
    # Investment figure
    graf5 = document.add_paragraph("In 2015, ", style="SEIA Bullet 1")
    graf5.add_run("$212 million was invested ").bold = True
    graf5.add_run("on solar installations in " + state + ".")

    # Average and national price comparisons
    graf6 = document.add_paragraph("Average installed residential and commercial photovoltaic system ", style="SEIA Bullet 1")
    graf6.add_run("prices in " + state + " have fallen by 4% in the last year.").bold = True
    graf6.add_run(" National Prices have also dropped steadily- by 4% from last year and 49% from 2010.")
    return


In [41]:
def notables(document, proj1, proj2, proj3):
    s2 = document.add_paragraph('Notable Projects', style="Section")

    proj1 = "Alamosa Solar Generating Project was completed in 2012 by developer Cogentrix.  This concentrating photovoltaics project has the capacity to generate 30 MW of electricity-- enough to power over 5,400 Colorado homes."
    p7 = document.add_paragraph(proj1, style="SEIA Bullet 1")

    proj2 = "At 50 MW, Hooper Solar Project is among the largest solar installations in Colorado.  Completed in 2016 by SunPower, this photovoltaic project has enough electric capacity to power more than 5,700 homes. "
    p8 = document.add_paragraph(proj2, style="SEIA Bullet 1")

    proj3 = "Several large retailers in Colorado have gone solar, including Kohl's, REI, Safeway and Walmart. Intel has installed one of the largest corporate photovoltaic systems in the state with 1000 kW of solar capacity at their location in Fort Collins. "
    p9 = document.add_paragraph(proj3, style="SEIA Bullet 1")
    return

In [42]:
def makeMap(document, data):
    url = 'https://api.mapbox.com/v4/mapbox.satellite/-76.9,38.9,5/1000x1000.png?access_token=pk.eyJ1IjoiYWFyb25sZWVoIiwiYSI6IjF0SjNqUUUifQ.0sVfP4L9LWoycJoinMovtA'
    response = requests.get(url, stream=True)
    fn = 'J:/SEIA/codebin/Factsheets/img.png'
    with open(fn, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response
    image = Image.open("J:/SEIA/codebin/Factsheets/charts/map_test.jpg")
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("C:/Windows/Fonts/verdana.ttf", 16)
    draw.text((10,10), "Colorado", (255, 128, 255), font=font)
    image.save("J:/SEIA/codebin/Factsheets/charts/map_test_output.jpg")
    fn2 = "J:/SEIA/codebin/Factsheets/charts/map_test_output.jpg"
    document.add_picture(fn2, width=Inches(7))
    return

In [43]:
def initdata():
    fp = "J:/SEIA/SMI/SMI_2016Q4_2.xlsx"

    #smi_cap_q = pd.read_excel(fp, sheetname="Capacity", skiprows=3, parse_cols='B:AE')
    #smi_cap_q.head()

    #smi_cap_a1 = pd.read_excel(fp, sheetname="Capacity", skiprows=3, parse_cols='B')
    #smi_cap_a2 = pd.read_excel(fp, sheetname="Capacity", skiprows=3, parse_cols='AG:AO')
    #smi_cap_a = smi_cap_a2.append(smi_cap_a1)
    #smi_cap_a.head()
    global cap_raw_total
    global cap_raw_res
    global cap_raw_nonres
    global cap_raw_utility
    #cap_raw.head()
    #cap_raw.drop('Q1 2010', axis=1, inplace=True)
    global cap_a_total
    global cap_a_res
    global cap_a_nonres
    global cap_a_utility
    #print(cap_a_total)
    #cap_raw_a.head()

    global cap_q_total
    global cap_q_res
    global cap_q_nonres
    global cap_q_utility
    global states
    global nsd
    
    cap_raw_total = pd.read_excel(fp, sheetname='Capacity', skiprows=3, parse_cols='B:AO', skip_footer=136)
    cap_raw_res = pd.read_excel(fp, sheetname='Capacity', skiprows=48, parse_cols='B:AO', skip_footer=91)
    cap_raw_nonres = pd.read_excel(fp, sheetname='Capacity', skiprows=93, parse_cols='B:AO', skip_footer=46)
    cap_raw_utility = pd.read_excel(fp, sheetname='Capacity', skiprows=138, parse_cols='B:AO', skip_footer=1)
    #cap_raw.head()
    #cap_raw.drop('Q1 2010', axis=1, inplace=True)
    cap_a_total = cap_raw_total.drop(cap_raw_total.columns[1:31], axis=1)
    cap_a_res = cap_raw_res.drop(cap_raw_res.columns[1:31], axis=1)
    cap_a_nonres = cap_raw_nonres.drop(cap_raw_nonres.columns[1:31], axis=1)
    cap_a_utility = cap_raw_utility.drop(cap_raw_utility.columns[1:31], axis=1)
    #print(cap_a_total)
    #cap_raw_a.head()

    cap_q_total = cap_raw_total.drop(cap_raw_total.columns[30:], axis=1)
    cap_q_res = cap_raw_res.drop(cap_raw_res.columns[30:], axis=1)
    cap_q_nonres = cap_raw_nonres.drop(cap_raw_nonres.columns[30:], axis=1)
    cap_q_utility = cap_raw_utility.drop(cap_raw_utility.columns[30:], axis=1)
    
    states = cap_raw_total['State']
    
    fp = "J:/SEIA/codebin/Factsheets/PrimaryCategories_20161130.csv"
    nsd = pd.read_csv(fp)
    
    return

In [99]:
#states = ['California', 'Massachusetts', 'Delaware']
initdata()
for state in states:
    # Init variables
    # Graf 1 - Jobs
    
    types = nsd[nsd['Physical State/Province'] == state]
    
    cos = types['Account Name'].count()
    #print(cos)
    #cos = types[types['Account Name']].count()
    jobs = 0 # need TSF
    
    mfrs_c = types[types['Primary Category'] == 'Manufacturer/Supplier']
    mfrs = mfrs_c['Primary Category'].count()
    mfr_facs = 0
    installers_c = types[(types['Primary Category'] == 'Contractor/Installer') | (types['Primary Category'] == 'EPC')]
    installers = installers_c['Primary Category'].count()
    devs_c = types[types['Primary Category'] == 'Project Developer']
    devs = devs_c['Primary Category'].count()
    distribs_c = types[types['Primary Category'] == 'Distributor']
    distribs = distribs_c['Primary Category'].count()
    others_c = types[(types['Primary Category'] == 'Consultant') 
                    | (types['Primary Category'] == 'Architectural/Engineering')
                    | (types['Primary Category'] == 'Communications/Marketing')
                    | (types['Primary Category'] == 'Corporate Solar User')
                    | (types['Primary Category'] == 'Educational Institution')
                    | (types['Primary Category'] == 'Electrical Utility') 
                    | (types['Primary Category'] == 'Financial Company')
                    | (types['Primary Category'] == 'Government' )
                    | (types['Primary Category'] == 'Legal Services') 
                    | (types['Primary Category'] == 'Need to Update Category')
                    | (types['Primary Category'] == 'None')
                    | (types['Primary Category'] == 'Non-Profit') 
                    | (types['Primary Category'] == 'Not for Profit') 
                    | (types['Primary Category'] == 'Partnership') 
                    | (types['Primary Category'] == 'Realtor')
                    | (types['Primary Category'] == 'Research') 
                    | (types['Primary Category'] == 'Service Provider') 
                    | (types['Primary Category'] == 'Sponsorship')
                    | (types['Primary Category'] == '')]
    
    others = others_c['Primary Category'].count()
    
    # Graf 2 - Quarterly
    # Quarterly total, residential, non-residential, utility capacities
    # q/q, y/y and quarterly rank
    q32016 = np.round(cap_q_total[cap_q_total['State'] == state]['Q3 2016'].values[0], decimals=2)
    q32016res = np.round(cap_q_res[cap_q_res['State'] == state]['Q3 2016'].values[0])
    q32016nonres = np.round(cap_q_nonres[cap_q_nonres['State'] == state]['Q3 2016'].values[0])
    q32016utility = np.round(cap_q_utility[cap_q_utility['State'] == state]['Q3 2016'].values[0])
    cap_q_total['rank'] = cap_q_total['Q3 2016'].rank(axis=0, ascending=False)
    # Q/Q
    q22016 = cap_q_total[cap_q_total['State'] == state]['Q2 2016'].values[0]
    qq = np.round((q32016-q22016)/q22016, decimals=2)
    
    # Y/Y
    q32015 = cap_q_total[cap_q_total['State'] == state]['Q3 2015'].values[0]
    yy = np.round((q32016-q32015)/q32015, decimals=2)
    
    # Rank
    rank_q = cap_q_total[cap_q_total['State'] == state]['rank'].values[0]
    
    #graf2s = state + ": " + str(q32016) + " Res: " + str(q32016res) + " Non-Res: " + str(q32016nonres) + " Utility: " + str(q32016utility)
    
    # Graf 3 - Annual
    # Annual total, residential, non-residential, utility capacities; rank
    c2015_total = cap_a_total[cap_a_total['State'] == state][2015].values[0]
    c2015_res = cap_a_res[cap_a_res['State'] == state][2015].values[0]
    c2015_nonres = cap_a_nonres[cap_a_nonres['State'] == state][2015].values[0]
    c2015_utility = cap_a_utility[cap_a_utility['State'] == state][2015].values[0]
    cap_a_total['rank'] = cap_a_total[2015].rank(axis=0, ascending=False)
    rank_a = cap_a_total[cap_a_total['State'] == state]['rank'].values[0]
    #print(row + " " + str(c2015_total) + " " + str(c2015_res) + " " + str(c2015_nonres) + " " + str(c2015_utility) + " " + str(rank_a))
    
    # Graf 4 - Cumulative
    # cumulative capacity (all sectors), rank, homes powered
    cumulative = cap_a_total[cap_a_total['State'] == state]['Cumulative'].values[0]
    cap_a_total['rank - cumulative'] = cap_a_total['Cumulative'].rank(axis=0, ascending=False)
    rank_a_cumulative = cap_a_total[cap_a_total['State'] == state]['rank - cumulative'].values[0]
    #print(row + " " + str(rank_a_cumulative))
    
    hp = 1234567
    
    template = 'J:/SEIA/codebin/Factsheets/state_fs_clean_template.docx'
    #template = 'J:/SEIA/codebin/Factsheets/NEWStatefactsheet.docx'
    doc = initdoc(template)

    graf1(doc, state, cos, jobs, mfrs, mfr_facs, installers, devs, distribs, others)

    # Add chart as picture from save location above, add to factsheet
    #wb_n = "J:/SEIA/codebin/Python/myworkbook.xlsx"
    wb_n = "J:/SEIA/codebin/Factsheets/workbench3.xlsx"
    stuff = [wb_n, state, cap_a_total[cap_a_total['State'] == state]]
    fn = makeChart(stuff)
    if (fn):
        doc.add_picture(fn, width=Inches(3))
    ranknq = rankTorank(rank_q)
    rankna = rankTorank(rank_a)
    ranknc = rankTorank(rank_a_cumulative)
    
    #graf2(doc, state, q32016, qq, yy, rank_q)
    #graf3(doc, state, c2015_total, c2015_res, c2015_nonres, c2015_utility, rank_a)
    #graf4(doc, state, cumulative, rank_a_cumulative, hp)
    graf2(doc, state, q32016, qq, yy, ranknq)
    graf3(doc, state, c2015_total, c2015_res, c2015_nonres, c2015_utility, rankna)
    graf4(doc, state, cumulative, ranknc, hp)
    othergrafs(doc, state, "stuff")

    notables(doc, "FLS1", "FLS2", "FLS3")

    makeMap(doc, "stuff")

    factsheet = "J:/SEIA/codebin/Factsheets/Q42016/test20161202_" + state + ".docx"
    doc.save(factsheet)

     State  Cumulative Pre 2010.1  2010  2011  2012  2013  2014     2015  \
0  Alabama                    NaN   NaN   NaN   NaN   NaN   NaN  0.40386   

      2016  Cumulative  rank  rank - cumulative  
0  1.07337         NaN  42.0                NaN  
     State  Cumulative Pre 2010.1  2010   2011   2012        2013    2014  \
1  Arizona                   47.6  63.0  289.5  718.6  420.693525  246.56   

        2015    2016   Cumulative  rank  rank - cumulative  
1  257.50065  228.34  2271.794175   6.0                3.0  
      State  Cumulative Pre 2010.1  2010  2011  2012  2013  2014       2015  \
2  Arkansas                    NaN   NaN   NaN   NaN   NaN   NaN  16.326437   

      2016  Cumulative  rank  rank - cumulative  
2  0.49954         NaN  27.0                NaN  
        State  Cumulative Pre 2010.1   2010   2011    2012     2013    2014  \
3  California                 712.32  215.8  577.1  1045.7  2620.81  3549.0   

     2015     2016  Cumulative  rank  rank - cumulat

In [ ]:
Architectural/Engineering
Communications/Marketing
Consultant
Contractor/Installer
Corporate Solar User
Distributor
Educational Institution
Electrical Utility
EPC
FinacialÂ Company
Financial Company
Government
Legal Services
Manufacturer/Supplier
Need to Update Category
None
Non-Profit
Not for Profit
Partnership
Project Developer
Realtor
Research
Service Provider
Sponsorship
(blank)


tenth
twentieth
third
thirty fifth


In [59]:
cap = cap_a_total[cap_a_total['State'] == 'California']
print(cap)

        State  Cumulative Pre 2010.1   2010   2011    2012     2013    2014  \
3  California                 712.32  215.8  577.1  1045.7  2620.81  3549.0   

     2015     2016  Cumulative  rank  rank - cumulative  
3  3267.9  3262.81    15251.44   2.0                1.0  


In [87]:
cap[2011].values[0]

577.10000000000002